In [1]:
# Initialize the random number generator
rng = MersenneTwister(2016);

In [2]:
# Size of matrix
n = 16
Q = rand(rng, n, n)
Λ = diagm(Float64[1.4^(-i) for i=0:n-1])

Q, = qr(Q)
Λ[n,n] = 1e-1 * Λ[n,n]

A = Q * Λ / Q

16×16 Array{Float64,2}:
  0.227973     0.166643    0.162037    …   0.05389     -0.0139736 
  0.166643     0.261279    0.129312        0.0397342    0.0400778 
  0.162037     0.129312    0.224327        0.03295      0.0412857 
  0.00297891  -0.0201759   0.0260055       0.00234127   0.00520295
 -0.0115157    0.0125484  -0.00921677      0.0764092    0.0275871 
  0.0159532   -0.0295952   0.0447301   …   0.10858      0.0700198 
  0.141001     0.177755    0.14943         0.0987644    0.0990724 
  0.0587557    0.0350338   0.0213561       0.09593      0.0196911 
  0.0789802    0.0548429   0.0982046       0.0520265    0.0954151 
  0.138705     0.0491052   0.118873        0.14555     -0.0414623 
  0.0361497    0.0156304  -0.00692833  …   0.119504     0.0461919 
 -0.0847394   -0.0233921  -0.0610421       0.142572     0.101279  
 -0.00441918  -0.0733711   0.0452529      -0.0183839    0.00965959
  0.00702074   0.0155854   0.0656713       0.0670086    0.0665951 
  0.05389      0.0397342   0.03295    

In [3]:
Tk = copy(A)
for k=1:10
    Uk, Rk = qr(Tk)
    Tk = Rk * Uk
end

# Print approx. and exact evalues
n_prt = 3
Tkdiag = diag(Tk)
exact = diag(Λ)
println("Tk    ",Tkdiag[1:n_prt])
println("Exact ",exact[1:n_prt])

Tk    [0.999816, 0.713679, 0.510782]
Exact [1.0, 0.714286, 0.510204]


In [4]:
# Create animation for QR iteration
using PyPlot
using PyCall
@pyimport matplotlib.animation as anim

ffmpeg_installed = true

if ffmpeg_installed && ( !isfile("QR_iteration.mp4") || !isfile("QR_iteration_unsym.mp4") )

    function record_snapshot(A, ims)
        """Saves figure snapshot for the animation"""
        im = imshow(copy(A), interpolation="nearest",
                    extent=(0.5,size(A,1)+0.5,size(A,2)+0.5,0.5))
        clim(-8,1)
        push!(ims, PyCall.PyObject[im])
    end

    fig = figure()
    ims = []

    Tk = copy(A)
    record_snapshot(log.(abs.(A))/log(10), ims)
    for k=1:70
        Uk, Rk = qr(Tk)
        Tk = Rk * Uk
        record_snapshot(log.(abs.(Tk))/log(10), ims) # This is used to make the movie
    end
    colorbar()
    ani = anim.ArtistAnimation(fig, ims, interval=100, blit=true)
    ani[:save]("QR_iteration.mp4", extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])
end

INFO: Precompiling module PyPlot.
INFO: Installing matplotlib via the Conda matplotlib package...


Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /Users/darve/.julia/v0.6/Conda/deps/usr:

The following NEW packages will be INSTALLED:

    backports.functools_lru_cache: 1.4-py27h2aca819_1   
    cycler:                        0.10.0-py27hfc73c78_0
    freetype:                      2.8-h143eb01_0       
    libpng:                        1.6.32-h6184301_3    
    matplotlib:                    2.1.0-py27h3078195_0 
    pytz:                          2017.2-py27hb891d23_1
    subprocess32:                  3.2.7-py27h24b2887_0 

The following packages will be UPDATED:

    zlib:                          1.2.8-3               --> 1.2.11-h60db283_1

zlib-1.2.11-h6 100% |###############################| Time: 0:00:00  12.62 MB/s
libpng-1.6.32- 100% |###############################| Time: 0:00:00  21.38 MB/s
freetype-2.8-h 100% |###############################| Time: 0:00:00  25.33 MB/s
pytz-2017.2-py 100% |###########

LoadError: InitError: [91mPyError (ccall(@pysym(:PyImport_ImportModule), PyPtr, (Cstring,), name)

The Python package mpl_toolkits.mplot3d could not be found by pyimport. Usually this means
that you did not install mpl_toolkits.mplot3d in the Python version being used by PyCall.

PyCall is currently configured to use the Julia-specific Python distribution
installed by the Conda.jl package.  To install the mpl_toolkits.mplot3d module, you can
use `pyimport_conda("mpl_toolkits.mplot3d", PKG)`, where PKG is the Anaconda
package the contains the module mpl_toolkits.mplot3d, or alternatively you can use the
Conda package directly (via `using Conda` followed by `Conda.add` etcetera).

Alternatively, if you want to use a different Python distribution on your
system, such as a system-wide Python (as opposed to the Julia-specific Python),
you can re-configure PyCall with that Python.   As explained in the PyCall
documentation, set ENV["PYTHON"] to the path/name of the python executable
you wan

In [5]:
function html_video(filename)
    open(filename) do f
        base64_video = base64encode(f)
        """<video controls src="data:video/x-m4v;base64,$base64_video">"""
    end
end

html_video (generic function with 1 method)

In [6]:
# Symmetric case
if isfile("QR_iteration.mp4") # Checks whether movie file exists
    display("text/html", html_video("QR_iteration.mp4"))
end

In [7]:
# Unsymmetric case
if isfile("QR_iteration_unsym.mp4") # Checks whether movie file exists
    display("text/html", html_video("QR_iteration_unsym.mp4"))
end